In [ ]:
%pip install ultralytics
import ultralytics

ultralytics.checks()

Ultralytics 8.3.144 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.3/107.7 GB disk)


In [ ]:
%pip install ultralytics flask

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving tests.mp4 to tests.mp4


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from datetime import datetime, timedelta
import json
import os

print("Files in /content:", os.listdir('/content'))

class SuspiciousTools:
    def __init__(self, static_threshold_seconds=300):
        self.immediate_threats = ['knife', 'gun', 'scissors']
        self.unattended_threats = ['backpack', 'suitcase', 'bottle']
        self.suspicious_classes = self.immediate_threats + self.unattended_threats
        self.alerts = []
        self.object_tracks = {}  # Dictionary to store tracks: {track_id: {'class': str, 'bbox': list, 'last_seen': datetime, 'first_seen': datetime}}
        self.static_threshold = timedelta(seconds=static_threshold_seconds)  # Time threshold for unattended objects
        self.position_threshold = 100  # Pixel threshold for considering an object static

    def is_suspicious(self, class_name):
        return class_name.lower() in [tool.lower() for tool in self.suspicious_classes]

    def is_immediate_threat(self, class_name):
        return class_name.lower() in [tool.lower() for tool in self.immediate_threats]

    def is_unattended_threat(self, class_name):
        return class_name.lower() in [tool.lower() for tool in self.unattended_threats]

    def generate_alert(self, class_name, bbox, frame_id, track_id=None, reason="Detected"):
        # Convert numpy types to Python native types for JSON serialization
        alert = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'tool': class_name,
            'location': {
                'x': float(bbox[0]),  # Convert numpy.float32 to float
                'y': float(bbox[1]),
                'width': float(bbox[2] - bbox[0]),
                'height': float(bbox[3] - bbox[1])
            },
            'frame_id': int(frame_id),  # Ensure frame_id is int
            'track_id': int(track_id) if track_id is not None else None,  # Convert track_id to int if not None
            'reason': reason
        }
        self.alerts.append(alert)
        return alert

    def update_tracks(self, class_name, bbox, track_id, frame_id):
        current_time = datetime.now()
        if self.is_unattended_threat(class_name):
            # Calculate centroid of the bounding box
            centroid = [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]

            if track_id not in self.object_tracks:
                # Initialize new track
                self.object_tracks[track_id] = {
                    'class': class_name,
                    'bbox': bbox,
                    'centroid': centroid,
                    'first_seen': current_time,
                    'last_seen': current_time,
                    'frame_id': frame_id
                }
            else:
                # Update existing track
                prev_centroid = self.object_tracks[track_id]['centroid']
                distance = np.sqrt((centroid[0] - prev_centroid[0])**2 + (centroid[1] - prev_centroid[1])**2)

                if distance < self.position_threshold:
                    # Object hasn't moved significantly
                    self.object_tracks[track_id]['last_seen'] = current_time
                    self.object_tracks[track_id]['bbox'] = bbox
                    self.object_tracks[track_id]['centroid'] = centroid
                    # Check if object has been static for too long
                    duration = current_time - self.object_tracks[track_id]['first_seen']
                    if duration >= self.static_threshold:
                        alert = self.generate_alert(
                            class_name=class_name,
                            bbox=bbox,
                            frame_id=frame_id,
                            track_id=track_id,
                            reason=f"Unattended for {duration.total_seconds():.0f} seconds"
                        )
                        print(f"Alert: {json.dumps(alert, indent=2)}")
                        # Remove track to avoid repeated alerts
                        del self.object_tracks[track_id]
                else:
                    # Object moved, reset track
                    self.object_tracks[track_id] = {
                        'class': class_name,
                        'bbox': bbox,
                        'centroid': centroid,
                        'first_seen': current_time,
                        'last_seen': current_time,
                        'frame_id': frame_id
                    }

    def get_alerts(self):
        return self.alerts

def process_video(video_path, model_path='yolov8l.pt', conf_threshold=0.05):
    # Check if video file exists
    if not os.path.exists(video_path):
        print(f"Error: Video file '{video_path}' does not exist.")
        return []

    try:
        file_size = os.path.getsize(video_path) / (1024 * 1024)  # Size in MB
        print(f"Video file size: {file_size:.2f} MB")
    except Exception as e:
        print(f"Error: Could not access video file. Details: {e}")
        return []

    # Initialize YOLO and SuspiciousTools
    try:
        model = YOLO(model_path)
    except Exception as e:
        print(f"Error: Could not load YOLO model. Details: {e}")
        return []

    suspicious_tools = SuspiciousTools(static_threshold_seconds=20)  # 2 minutes for unattended objects

    # Open video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video '{video_path}'. Check file format or corruption.")
        return []

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print(f"Video properties: {frame_width}x{frame_height}, {fps} FPS")

    # Initialize video writer
    out = cv2.VideoWriter('/content/output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    frame_id = 0
    detected_classes = set()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform detection with tracking
        results = model.track(frame, conf=conf_threshold, persist=True, verbose=False)

        # Process detections
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy()
            confidences = result.boxes.conf.cpu().numpy()
            track_ids = result.boxes.id.cpu().numpy() if result.boxes.id is not None else [None] * len(boxes)
            names = result.names

            for box, cls, conf, track_id in zip(boxes, classes, confidences, track_ids):
                class_name = names[int(cls)]
                detected_classes.add(class_name)
                if suspicious_tools.is_suspicious(class_name):
                    if suspicious_tools.is_immediate_threat(class_name):
                        # Immediate alert for knives, guns, scissors
                        alert = suspicious_tools.generate_alert(class_name, box, frame_id, track_id)
                        print(f"Alert: {json.dumps(alert, indent=2)}")
                    else:
                        # Update track for unattended threats
                        suspicious_tools.update_tracks(class_name, box, track_id, frame_id)

                    # Draw bounding box
                    x1, y1, x2, y2 = map(int, box)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    label = f"{class_name} {conf:.2f} ID:{track_id}"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        out.write(frame)
        frame_id += 1

    cap.release()
    out.release()

    print(f"\nDetected classes: {sorted(detected_classes)}")
    return suspicious_tools.get_alerts()

def simulate_web_app(alerts):
    print("\n=== Web App Alert Simulation ===")
    if not alerts:
        print("No suspicious objects detected.")
    for alert in alerts:
        print(f"Alert at {alert['timestamp']}:")
        print(f"  Tool: {alert['tool']}")
        print(f"  Reason: {alert['reason']}")
        print(f"  Location: X={alert['location']['x']:.2f}, Y={alert['location']['y']:.2f}, "
              f"Width={alert['location']['width']:.2f}, Height={alert['location']['height']:.2f}")
        print(f"  Frame ID: {alert['frame_id']}")
        if alert['track_id'] is not None:
            print(f"  Track ID: {alert['track_id']}")
        print("")

# Run detection and simulation
if __name__ == "__main__":
    video_path = "/content/tests.mp4"  # Correct file name
    alerts = process_video(video_path, model_path='yolo11n.pt', conf_threshold=0.05)
    simulate_web_app(alerts)

Files in /content: ['.config', 'output.mp4', 'alert.mp4', 'yolo11n.pt', 'tests.mp4', 'runs']
Video file size: 97.92 MB
Video properties: 1376x774, 30 FPS
Alert: {
  "timestamp": "2025-05-24 13:31:25",
  "tool": "backpack",
  "location": {
    "x": 554.2921752929688,
    "y": 450.6133728027344,
    "width": 48.8284912109375,
    "height": 74.75222778320312
  },
  "frame_id": 119,
  "track_id": 50,
  "reason": "Unattended for 20 seconds"
}
Alert: {
  "timestamp": "2025-05-24 13:32:06",
  "tool": "backpack",
  "location": {
    "x": 715.7329711914062,
    "y": 446.1653137207031,
    "width": 45.61065673828125,
    "height": 80.86599731445312
  },
  "frame_id": 301,
  "track_id": 261,
  "reason": "Unattended for 20 seconds"
}
Alert: {
  "timestamp": "2025-05-24 13:43:01",
  "tool": "suitcase",
  "location": {
    "x": 510.91143798828125,
    "y": 315.93023681640625,
    "width": 296.02862548828125,
    "height": 363.9537353515625
  },
  "frame_id": 3214,
  "track_id": 4718,
  "reason": "Un

In [ ]:
!yolo track source="/content/tests.mp4" save=True

WARNING ⚠️ 'model' argument is missing. Using default 'model=yolo11n.pt'.
Ultralytics 8.3.144 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

video 1/1 (frame 1/4592) /content/tests.mp4: 384x640 6 persons, 3 cars, 1 motorcycle, 4 trucks, 292.6ms
video 1/1 (frame 2/4592) /content/tests.mp4: 384x640 6 persons, 3 cars, 1 motorcycle, 4 trucks, 246.7ms
video 1/1 (frame 3/4592) /content/tests.mp4: 384x640 6 persons, 3 cars, 1 motorcycle, 4 trucks, 220.9ms
video 1/1 (frame 4/4592) /content/tests.mp4: 384x640 6 persons, 3 cars, 1 motorcycle, 4 trucks, 240.3ms
video 1/1 (frame 5/4592) /content/tests.mp4: 384x640 6 persons, 3 cars, 1 motorcycle, 4 trucks, 239.8ms
video 1/1 (frame 6/4592) /content/tests.mp4: 384x640 5 persons, 3 cars, 1 motorcycle, 4 trucks, 241.2ms
video 1/1 (frame 7/4592) /content/tests.mp4: 384x640 4 persons, 2 cars, 1 motorcycle, 5 trucks, 162.0ms
video 1/1 (frame 8/4592) /content/